In [ ]:
# default_exp priceHist

# priceHist

> Parse historical stock price data files.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

import os
import pandas as pd

from secscan import utils

USExchanges = ['AMEX','NASDAQ','NYSE','OTCBB']

Uses the historical price data from eoddata.com -
this comes as one CSV file per exchange per day, sample:

```
Symbol,Date,Open,High,Low,Close,Volume
AACG,08-Jan-2020,1.41,1.58,1.3642,1.5112,90800
AAL,08-Jan-2020,27.1,28.09,27.07,27.84,10497200
...
```


In [ ]:
#export

def getHistFStuff(exch,dateStr) :
    """
    Returns (fName, fDir, fPath) containing the CSV file of historical prices
    for the given exchange and date.
    """
    fName = exch+'_'+dateStr+'.csv'
    histFDir = os.path.join(utils.stockPriceRoot,fName[:len(exch)+5])
    histFPath = os.path.join(histFDir,fName)
    return fName,histFDir,histFPath

In [ ]:
# test getHistFStuff:

t = getHistFStuff('NASDAQ','20240510')
assert (t[0]=='NASDAQ_20240510.csv'
        and t[1]==os.path.join(utils.stockPriceRoot,'NASDAQ_2024')
        and t[2]==os.path.join(utils.stockPriceRoot,'NASDAQ_2024',t[0]))

In [ ]:
#export

def getDayMap(dateStr, exch, symCol='Symbol', priceCol='Close') :
    """
    Parse the CSV file for a single day and exchange,
    returning a dict:<stock symbol> -> <value>
    """
    fPath = getHistFStuff(exch,dateStr)[2]
    if not os.path.exists(fPath) :
        return {}
    df = pd.read_csv(fPath)
    return dict(zip(df[symCol],df[priceCol]))

In [ ]:
# test getDayMap:

utils.setStockPriceRoot('testdata')

m = getDayMap('20200108','AMEX')
assert (len(m)==2024
        and min(m.keys())=='AAAU' and m['AAAU']==15.56
        and max(m.keys())=='ZSL' and m['ZSL']==26.04)

m = getDayMap('20200108','NASDAQ')
assert (len(m)==3233
        and min(m.keys())=='AACG' and m['AACG']==1.5112
        and max(m.keys())=='ZYXI' and m['ZYXI']==8.47)

In [ ]:
#export

@utils.delegates(getDayMap)
def getCombDayMap(dateStr, exchs=USExchanges, checkDups=True, **kwargs) :
    """
    Combines the day maps for a list of exchanges, optionally checking for symbols
    duplicated between exchanges.
    """
    res = {}
    for exch in exchs :
        m = getDayMap(dateStr,exch,**kwargs)
        if checkDups :
            dupKeys = (set(m.keys()) & res.keys())
            if len(dupKeys) > 0 :
                print('duplicated keys:',exch,dateStr,sorted(dupKeys)[:10])
        res.update(m)
    return res

In [ ]:
# test getCombDayMap:

m = getCombDayMap('20200108',['AMEX','NASDAQ'])
assert len(m)==2024+3233

In [ ]:
#hide
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()